
<img width="60" src="https://drive.google.com/uc?export=view&id=1JQRWCUpJNAvselJbC_K5xa5mcKl1gBQe"> 



In [1]:
# Uploading files from your local file system

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving edunatal.csv to edunatal.csv
User uploaded file "edunatal.csv" with length 14817 bytes


## Import the dataset and verify basic information

- **NAME** - name of school
- **Type** -  provides information about public elementary schools and kindergartens of Natal-RN in 2014.
- **GEOCODE_INPUT** - school's address
- **LAT** - latitute coordinate of school
- **LNG** - longitude coordinate of school




In [2]:
import pandas as pd

# read the dataset and delete the first columns
data = pd.read_csv("edunatal.csv").iloc[:,1:].dropna().reset_index(drop=True)
data.head()

,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,ESC MUL PROF ANTÔNIO SEVERIANO,Elem,"AV OURO PRETO, 2754, NATAL-RN",-5.870580,-35.214868
1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,Elem,"ARACITABA, 2993, NATAL-RN",-5.868884,-35.197566
2,ESC MUL PROF ASCENDINO DE ALMEIDA,Elem,"RUA JOAQUIM CARDOSO, , NATAL-RN",-5.851115,-35.240676
3,ESC MUL PROF CARLOS BELLO MORENO,Elem,"RUA ARAPIRACA, , NATAL-RN",-5.863614,-35.207138
4,ESC MUL PROF OTTO DE BRITO GUERRA,Elem,"RUA SERRA DA JUREMA, , NATAL-RN",-5.855495,-35.245909


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 5 columns):
NAME             144 non-null object
Type             144 non-null object
GEOCODE_INPUT    144 non-null object
LAT              144 non-null float64
LNG              144 non-null float64
dtypes: float64(2), object(3)
memory usage: 5.7+ KB


## Preparing the data

In [4]:
# install folium packages
!pip install folium geocoder

    100% |████████████████████████████████| 81kB 2.6MB/s 
    100% |████████████████████████████████| 102kB 5.0MB/s 
    100% |████████████████████████████████| 81kB 5.4MB/s 


In [0]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import folium
from folium.plugins import HeatMap
import branca
from folium import plugins
import geocoder

In [6]:
# Calculate the geolocation coordenates 
for i in tqdm(range(len(data))):
    g = geocoder.arcgis(data.loc[i,'GEOCODE_INPUT'])
    data.loc[i,'LAT'] = g.lat
    data.loc[i,'LNG'] = g.lng

100%|██████████| 144/144 [01:11<00:00,  2.34it/s]


In [7]:
data.head()

,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,ESC MUL PROF ANTÔNIO SEVERIANO,Elem,"AV OURO PRETO, 2754, NATAL-RN",-5.871020,-35.214507
1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,Elem,"ARACITABA, 2993, NATAL-RN",-5.869254,-35.197306
2,ESC MUL PROF ASCENDINO DE ALMEIDA,Elem,"RUA JOAQUIM CARDOSO, , NATAL-RN",-5.850180,-35.240096
3,ESC MUL PROF CARLOS BELLO MORENO,Elem,"RUA ARAPIRACA, , NATAL-RN",-5.863684,-35.207176
4,ESC MUL PROF OTTO DE BRITO GUERRA,Elem,"RUA SERRA DA JUREMA, , NATAL-RN",-5.855496,-35.245897


## Data Analysis 

In [0]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Dictionary for colors
unit_type_colors = {
    'Elem': 'green',
    'KGAR': 'red',
}

# Dictionary for icons
unit_type_icons = {
    'Elem': 'graduation-cap',
    'KGAR': 'odnoklassniki',
}

In [9]:
# Ploting the educational units on map
for i in tqdm(range(len(data))):
    folium.Marker([data.loc[i,'LAT'], data.loc[i,'LNG']],
                  icon=folium.Icon(
                          color = unit_type_colors[data.loc[i,'Type']],
                          icon = unit_type_icons[data.loc[i,'Type']],
                          prefix='fa'),
                  popup = data.loc[i, 'NAME']
        ).add_to(m)

m

100%|██████████| 144/144 [00:00<00:00, 5118.60it/s]


In [10]:
coordinates_elem = []
coordinates_kgarten = []

for i in tqdm(range(len(data))):
    if (data.loc[i,'Type'] == 'Elem'):
        coordinates_elem.append([data.loc[i,'LAT'], data.loc[i,'LNG']])
    else:
        coordinates_kgarten.append([data.loc[i,'LAT'], data.loc[i,'LNG']])
        
# Create a map object
m_elem = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Create a map object
m_kgarten = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Create a heat map
HeatMap(coordinates_elem).add_to(m_elem)
HeatMap(coordinates_kgarten).add_to(m_kgarten)


100%|██████████| 144/144 [00:00<00:00, 17716.69it/s]


In [11]:
f = branca.element.Figure()

d1 = f.add_subplot(1, 2, 1)
d2 = f.add_subplot(1, 2, 2)

d1.add_child(m_elem)
d2.add_child(m_kgarten)